In [2]:
import numpy as np
import pandas as pd
import networkx as nx

In [3]:
'''
Calculates the sum of the weights of an edgeslist of the bipartite graph
Takes in a list of edges (the ones adjacent to the user or the song.)
'''

def cal_adj_weights(edges):
    edge_data = [G.get_edge_data(e[0],e[1]) for e in edges]
    total_weight = 0.0
    for i in edge_data:
        total_weight += i['weight']
    return(total_weight)

'''
Finds the average number of song listens by a user u.
bipart_g: is a weighted bipartite networkx Graph, where the weight = # of listens of song s by user u
It returns 1 / |S_u| Σ r_(u,s) => the average listens for a user u
'''

def mean_ru(user_u, bipart_g: nx.Graph):
    edges_u = [e for e in bipart_g.edges(user_u)]
    song_count = len(edges_u) # R_u
    listen_count = cal_adj_weights(edges)
    return (listen_count / song_count)


'''
Finds the 2-step random walk recommendation power for an arbitrary user v from a choosen user u
We know both users u,v
This is P(u -> s -> v), as this is a bipartite graph
v is always 2 steps from u
P(Transition from user u to song s) = r_(u,s) / R_u

P(Transition from song s to user v) = r_(v,s) / R_s
'''
def rec_power_u_v(user_u, user_v, bipart_g: nx.Graph):
    edges_u = [e for e in bipart_g.edges(user_u)]
    edges_v = [e for e in bipart_g.edges(user_v)]
    songs_u = [n for n in bipart_g.neighbors(user_u)]
    
    song_count_u = len(edges_u) 
    listen_count_u = cal_adj_weights(edges_u) # R_u
    
    running_value = 0.0
    
    for s in songs_u:
        users_s = [e for e in bipart_g.edges(song)] # Users that listened to song s
        listen_count_s = cal_adj_weights(users_s) # R_s
        
        if bipart_g.has_edge(s, user_v):
            user_v_listens_s = bipart_g.get_edge_data(user_v, s)['weight'] # r_v,s
            user_u_listens_s = bipart_g.get_edge_data(user_u, s)['weight'] # r_u,s
            running_value += user_v_listens_s * user_u_listens_s / listen_count_s
    return (running_value / listen_count_u)



'''
Predicts the number of listens that user u will give to song s
'''
def predict_rating(user_u, song_s, total_users, bipart_g: nx.Graph):
    u_bar = mean_ru(user_u, bipart_g)
    v_bar = mean_ru(user_v, bipart_g)
    running_value = 0.0
    for v in total_users:
        if bipart_g.has_edge(song_s, v):
            user_v_listens_s = bipart_g.get_edge_data(v, s)['weight']
            rp_u_v = rec_power_u_v(user_u, user_v, bipart_g)
            running_value += (rp_u_v * (user_v_listens_s - v_bar))
    return u_bar + running_value

In [4]:
G = nx.Graph()
G.add_edge('a', 'b', weight=3) 
G.add_edge('b', 'c', weight=4)
G.add_edge('a', 'c', weight=1)

In [5]:
edges = [e for e in G.edges('a')]
print(len(edges))
edge_data = [G.get_edge_data(e[0],e[1]) for e in edges]
total_weight = 0.0
for i in edge_data:
    total_weight += i['weight']
print(total_weight)

2
4.0


In [6]:
cal_adj_weights(edges)

4.0

In [6]:
mean_ru('a', G)

2.0

In [7]:
df = pd.read_csv('../data/selected_triplet_new.csv') 
df

,user,song,weight
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBVFZR12A6D4F8AE3,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOEGVZY12A58A7857E,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOEKWEA12A6D4F5DC3,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOEOBYG12A6D4F8AE2,1
...,...,...,...
5354952,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,7
5354953,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOHHKGO12AC3DF57BF,1
5354954,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SONWLIS12A8C140865,2
5354955,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOQBOWE12A8C13CC2E,1


In [27]:
BiPart_G = nx.Graph()

for idx, row in df.iterrows():
    if idx % 100000 == 0: print(idx)
    if idx == 500000: break
    u = row['user']
    s = row['song']
    BiPart_G.add_node(u)
    BiPart_G.add_node(s)
    BiPart_G.add_edge(u, s, weight = int(row['weight']))


0
100000
200000
300000
400000
500000


In [30]:
predict_rating('b80344d063b5ccb3212f76538f3d9e43d87dca9e', 'SOTNMFD12A58A7789E', total_users = 500000, bipart_g = BiPart_G)

TypeError: predict_rating() missing 1 required positional argument: 'total_users'